In [31]:
import nflgame
import pandas as pd
import numpy as np

# Defensive Stats Dataframe

In [53]:
teams_map = {}
teams = []
for team in nflgame.teams:
    teams.append(team[0])
    for i in range(len(team)):
        teams_map[team[i]] = team[0]

In [57]:
# Build DataFrame - Index for Player name and week #
wks = range(1,18)
idxs = [teams, wks]

idx = pd.MultiIndex.from_product(idxs, names=['name', 'week'])
columns = [
    'rushing_att', 
    'rushing_yds', 
    'rushing_tds', 
    'fumbles_tot',
    'DNP',
    'fantasy_points'
]
df = pd.DataFrame(data=None, index=idx, columns=columns)

In [58]:
df.head(5)

rushing_att rushing_yds rushing_tds fumbles_tot  DNP fantasy_points
name week                                                                    
ARI  1            NaN         NaN         NaN         NaN  NaN            NaN
     2            NaN         NaN         NaN         NaN  NaN            NaN
     3            NaN         NaN         NaN         NaN  NaN            NaN
     4            NaN         NaN         NaN         NaN  NaN            NaN
     5            NaN         NaN         NaN         NaN  NaN            NaN

# Get Data

In [2]:
# NFL Games for 2016
games = nflgame.games(2016)

In [63]:
for g in games:
    wk = g.schedule['week']
    rushing_att_h = 0
    rushing_yds_h = 0
    fumbles_tot_h = 0
    rushing_tds_h = 0
    
    rushing_att_a = 0
    rushing_yds_a = 0
    fumbles_tot_a = 0
    rushing_tds_a = 0
    for p in g.players.rushing():
        if p.home:
            rushing_att_h += p.rushing_att
            rushing_yds_h += p.rushing_yds
            fumbles_tot_h += p.fumbles_tot
            rushing_tds_h += p.rushing_tds
        else:
            rushing_att_a += p.rushing_att
            rushing_yds_a += p.rushing_yds
            fumbles_tot_a += p.fumbles_tot
            rushing_tds_a += p.rushing_tds
    
    # Home Defense accumulates Away Offense
    df.loc[(g.home, wk), 'rushing_att'] = rushing_att_a
    df.loc[(g.home, wk), 'rushing_yds'] = rushing_yds_a
    df.loc[(g.home, wk), 'fumbles_tot'] = fumbles_tot_a
    df.loc[(g.home, wk), 'rushing_tds'] = rushing_tds_a
    
    # Away Defense accumulates Home Offense
    df.loc[(g.away, wk), 'rushing_att'] = rushing_att_h
    df.loc[(g.away, wk), 'rushing_yds'] = rushing_yds_h
    df.loc[(g.away, wk), 'fumbles_tot'] = fumbles_tot_h
    df.loc[(g.away, wk), 'rushing_tds'] = rushing_tds_h

In [65]:
# Calculate Fantasy Points
df['fantasy_points'] = df['rushing_yds'] * .1 + df['rushing_tds'] * 6 - df['fumbles_tot'] * 2

In [66]:
# DNP
df.loc[df['fantasy_points'].isnull(), 'DNP'] = True
df.loc[df['fantasy_points'].notnull(), 'DNP'] = False

# Fill NaNs
df = df.fillna(0)

# Boolean Values to int
df['DNP'] = df['DNP'].astype('int')

In [68]:
df.head(17)

rushing_att  rushing_yds  rushing_tds  fumbles_tot  DNP  \
name week                                                            
ARI  1              31          106            1            3    0   
     2              21           85            0            1    0   
     3              32          208            3            3    0   
     4              23           41            0            1    0   
     5              36          151            2            0    0   
     6              14           33            0            1    0   
     7              19           52            0            2    0   
     8              36          141            2            1    0   
     9               0            0            0            0    1   
     10             28           83            1            0    0   
     11             24           72            1            1    0   
     12             30          116            3            0    0   
     13             18           87            1            1    0   
     14             31           83            0            4    0   
     15             32          130            2            1    0   
     16             27           78            0            1    0   
     17             22           52            0            3    0   

           fantasy_points  
name week                  
ARI  1               10.6  
     2                6.5  
     3               32.8  
     4                2.1  
     5               27.1  
     6                1.3  
     7                1.2  
     8               24.1  
     9                0.0  
     10              14.3  
     11              11.2  
     12              29.6  
     13              12.7  
     14               0.3  
     15              23.0  
     16               5.8  
     17              -0.8

In [72]:
df.expanding().mean().astype(int)

rushing_att  rushing_yds  rushing_tds  fumbles_tot  DNP  \
name week                                                            
ARI  1              31          106            1            3    0   
     2              26           95            0            2    0   
     3              28          133            1            2    0   
     4              26          110            1            2    0   
     5              28          118            1            1    0   
     6              26          104            1            1    0   
     7              25           96            0            1    0   
     8              26          102            1            1    0   
     9              23           90            0            1    0   
     10             24           90            0            1    0   
     11             24           88            0            1    0   
     12             24           90            1            1    0   
     13             24           90            1            1    0   
     14             24           89            1            1    0   
     15             25           92            1            1    0   
     16             25           91            1            1    0   
     17             24           89            0            1    0   
ATL  1              25           89            0            1    0   
     2              25           92            0            1    0   
     3              25           93            0            1    0   
     4              24           91            0            1    0   
     5              24           91            0            1    0   
     6              24           90            0            1    0   
     7              24           90            0            1    0   
     8              24           90            0            1    0   
     9              24           90            0            1    0   
     10             24           94            0            1    0   
     11             23           91            0            1    0   
     12             23           89            0            1    0   
     13             23           91            0            1    0   
...                ...          ...          ...          ...  ...   
WAS  3              22           96            0            0    0   
     4              22           96            0            0    0   
     5              22           96            0            0    0   
     6              22           96            0            0    0   
     7              22           96            0            0    0   
     8              22           96            0            0    0   
     9              22           96            0            0    0   
     10             22           96            0            0    0   
     11             22           96            0            0    0   
     12             22           96            0            0    0   
     13             22           96            0            0    0   
     14             22           96            0            0    0   
     15             22           96            0            0    0   
     16             22           96            0            0    0   
     17             22           96            0            0    0   
JAX  3              22           96            0            0    0   
     4              22           96            0            0    0   
     7              23           96            0            0    0   
     10             23           96            0            0    0   
     13             23           96            0            0    0   
     14             23           96            0            0    0   
     16             23           96            0            0    0   
     2              23           96            0            0    0   
     6              23           96            0            0    0   
     8              2